In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE139038"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE139038"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE139038.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE139038.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE139038.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling in paired normal, apparently normal and breast tumour tissues"
!Series_summary	"The main objective of the study was to identify potential diagnostic and follow up markers along with therapeutic targets for breast cancer. We performed gene expression studies using the microarray technology on 65 samples including 41 breast tumours [24 early stage, 17 locally advanced, 18 adjacent normal tissue [paired normal] and 6 apparently normal from breasts which had been operated for non-malignant conditions. All the samples had frozen section done – tumours needed to have 70% or more tumour cells; paired normal and apparently normal had to be morphologically normal with no tumour cells."
!Series_overall_design	"Two-dye experiments using Universal Control RNA (Stratagene) and RNA from tissues."
!Series_overall_design	"Biological replicates - Apparently normal = 6, Paired normal = 18, Breast tumor tissues = 41"
Sample Characteristics 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Determine gene expression data availability
# Based on the background information, this dataset contains gene expression data for breast cancer
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Looking at row 3 which contains 'cancer stage' to distinguish between normal and tumor samples
trait_row = 3  # 'cancer stage' row

# Age: Available in row 0
age_row = 0  # 'age' row

# Gender: All samples are female (row 1 shows "gender: Female" with no variation)
# This is a constant feature, so not useful for association study
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert cancer stage information to binary trait (0 for normal, 1 for cancer)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Determine if the sample is normal or cancer
    if 'normal' in value.lower() or 'apparent normal' in value.lower():
        return 0  # Normal tissue
    elif 't' in value.lower() and any(x in value.lower() for x in ['n', 'm']):
        return 1  # Cancer tissue
    else:
        return None  # Unknown

def convert_age(value):
    """Convert age information to a continuous numeric value"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)  # Convert to float (continuous variable)
    except (ValueError, TypeError):
        return None  # Invalid or non-numeric age

def convert_gender(value):
    """Convert gender information to binary (0 for female, 1 for male)"""
    # Not used in this dataset as all samples are female
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None

# Initial validation
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data was obtained in a previous step and contains the sample characteristics
    # In this case, we need to create a DataFrame from the sample characteristics dictionary
    
    # Convert the sample characteristics dictionary to a DataFrame
    characteristics = {}
    for row_idx, values in sample_characteristics_dict.items():
        characteristics[row_idx] = values
    
    clinical_data = pd.DataFrame.from_dict(characteristics, orient='index').T
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [nan, 35.0], 1: [nan, nan], 2: [nan, nan], 3: [1.0, nan], 4: [nan, nan], 5: [1.0, nan], 6: [nan, nan], 7: [nan, nan], 8: [1.0, nan]}
Clinical data saved to ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE139038.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 80
Header line: "ID_REF"	"GSM4127905"	"GSM4127906"	"GSM4127907"	"GSM4127908"	"GSM4127909"	"GSM4127910"	"GSM4127911"	"GSM4127912"	"GSM4127913"	"GSM4127914"	"GSM4127915"	"GSM4127916"	"GSM4127917"	"GSM4127918"	"GSM4127919"	"GSM4127920"	"GSM4127921"	"GSM4127922"	"GSM4127923"	"GSM4127924"	"GSM4127925"	"GSM4127926"	"GSM4127927"	"GSM4127928"	"GSM4127929"	"GSM4127930"	"GSM4127931"	"GSM4127932"	"GSM4127933"	"GSM4127934"	"GSM4127935"	"GSM4127936"	"GSM4127937"	"GSM4127938"	"GSM4127939"	"GSM4127940"	"GSM4127941"	"GSM4127942"	"GSM4127943"	"GSM4127944"	"GSM4127945"	"GSM4127946"	"GSM4127947"	"GSM4127948"	"GSM4127949"	"GSM4127950"	"GSM4127951"	"GSM4127952"	"GSM4127953"	"GSM4127954"	"GSM4127955"	"GSM4127956"	"GSM4127957"	"GSM4127958"	"GSM4127959"	"GSM4127960"	"GSM4127961"	"GSM4127962"	"GSM4127963"	"GSM4127964"	"GSM4127965"	"GSM4127966"	"GSM4127967"	"GSM4127968"	"GSM4127969"
First data line: "10_10_1"	-0.943	0.844	0.14	0.566	-1.208	-0.216	0.036	0.136	0.542	0.205	0.848	-0.322	-0

Index(['10_10_1', '10_10_10', '10_10_11', '10_10_12', '10_10_13', '10_10_14',
       '10_10_15', '10_10_16', '10_10_17', '10_10_18', '10_10_19', '10_10_2',
       '10_10_20', '10_10_21', '10_10_22', '10_10_23', '10_10_24', '10_10_25',
       '10_10_26', '10_10_27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown (like "10_10_1", "10_10_10", etc.) are not standard human gene symbols.
# Standard human gene symbols would typically be recognizable names like "CFTR", "TP53", etc.
# These appear to be probe or feature identifiers from the microarray platform that need mapping.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_1_1', '1_2_1', '1_3_1', '1_4_1', '1_5_1'], 'Block': ['1', '1', '1', '1', '1'], 'Column': [1.0, 2.0, 3.0, 4.0, 5.0], 'Row': [1.0, 1.0, 1.0, 1.0, 1.0], 'description': ['Actin, beta', 'Actin, beta', 'Ribosomal protein L13a', 'Actin, beta', 'Hypoxanthine phosphoribosyltransferase 1 (Lesch-Nyhan syndrome)'], 'Gene_Symbol': ['ACTB', 'ACTB', 'RPL13A', 'ACTB', 'HPRT1'], 'SPOT_ID': ['hSQ025392', 'hSQ025392', 'hSQ018331', 'hSQ025392', 'hSQ021313']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation correspond to gene IDs and gene symbols
# Looking at the data preview, we can see:
# - The 'ID' column in gene_annotation contains identifiers like '1_1_1'
# - The gene expression data has row IDs like '10_10_1'
# - The 'Gene_Symbol' column contains standard gene symbols like 'ACTB'

# Since both datasets use the 'ID' column for identifiers, and 'Gene_Symbol' contains the gene symbols:
prob_col = 'ID'
gene_col = 'Gene_Symbol'

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {list(gene_data.index[:10])}")

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['1_1_1', '1_2_1', '1_3_1', '1_4_1', '1_5_1'], 'Gene': ['ACTB', 'ACTB', 'RPL13A', 'ACTB', 'HPRT1']}


Gene expression data after mapping:
Shape: (18744, 65)
First few gene symbols: ['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC']


Gene expression data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE139038.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was saved in a previous step
try:
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Clinical data shape:", clinical_data.shape)
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # The clinical data appears to have numeric column names, not sample IDs
    # Need to reconstruct clinical data to use proper sample IDs
    
    # Get sample IDs from gene expression data
    sample_ids = normalized_gene_data.columns
    
    # Reconstruct clinical data with appropriate structure
    print("Sample IDs from gene expression data:", sample_ids[:5], "...")
    
    # Get raw clinical data again from the original matrix file
    _, clinical_raw = get_background_and_clinical_data(matrix_file)
    
    # Verify raw clinical data has sample IDs
    print("Raw clinical data columns:", clinical_raw.columns[:5], "...")
    
    # Extract trait information using already defined conversion functions
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Rebuilt clinical features:")
    print(clinical_features.head())
    
    # 3. Link clinical and genetic data properly
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    print(f"  Genes with >20% missing: {sum(linked_data.iloc[:, 3:].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data.iloc[:, 3:].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data of breast cancer vs normal samples. Despite the cohort being in the Cystic_Fibrosis directory, the content is actually related to breast cancer."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: Dataset seems to be about breast cancer, not cystic fibrosis. Processing failed with error: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (17356, 65)
First few genes with their expression values after normalization:
        GSM4127905  GSM4127906  GSM4127907  GSM4127908  GSM4127909  \
Gene                                                                 
A1BG         1.121       0.642       0.495       0.979      -1.178   
A2M          3.808       1.866       2.066       2.439       2.413   
A2ML1        1.433       0.974       0.905       0.688       0.807   
A4GALT      -0.253       0.835       1.503       0.943      -0.485   
A4GNT       -0.617      -0.231       1.585      -0.133       0.348   

        GSM4127910  GSM4127911  GSM4127912  GSM4127913  GSM4127914  ...  \
Gene                                                                ...   
A1BG        -0.152       0.530       1.520       0.769       0.709  ...   
A2M          1.896       1.527       1.776       1.970       0.804  ...   
A2ML1       -0.074      -0.170       0.500       0.597       0.766  ...   
A4GALT       0.117       1.1

Normalized gene data saved to ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE139038.csv
Clinical data shape: (2, 9)
Clinical data preview:
                    0   1   2    3   4    5   6   7    8
Cystic_Fibrosis   NaN NaN NaN  1.0 NaN  1.0 NaN NaN  1.0
Age              35.0 NaN NaN  NaN NaN  NaN NaN NaN  NaN
Sample IDs from gene expression data: Index(['GSM4127905', 'GSM4127906', 'GSM4127907', 'GSM4127908', 'GSM4127909'], dtype='object') ...
Raw clinical data columns: Index(['!Sample_geo_accession', 'GSM4127905', 'GSM4127906', 'GSM4127907',
       'GSM4127908'],
      dtype='object') ...
Rebuilt clinical features:
                 GSM4127905  GSM4127906  GSM4127907  GSM4127908  GSM4127909  \
Cystic_Fibrosis         0.0         0.0         0.0         0.0         0.0   
Age                    35.0        67.0        35.0        36.0        40.0   

                 GSM4127910  GSM4127911  GSM4127912  GSM4127913  GSM4127914  \
Cystic_Fibrosis         0.0         1.0         1.0    

Data shape after handling missing values: (65, 17358)
For the feature 'Cystic_Fibrosis', the least common label is '0.0' with 24 occurrences. This represents 36.92% of the dataset.
The distribution of the feature 'Cystic_Fibrosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 45.0
  50% (Median): 54.0
  75%: 60.0
Min: 31.0
Max: 75.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cystic_Fibrosis/GSE139038.csv
